# Estimating reach curves from one data point

## Abstract

Reach Curves 能一定程度揭示一个广告的曝光数和观看这个广告的人数之间的关系，可以衡量广告在 TV 和网站投放的效果。实际中很难获得 reach curve 的完整曲线，advertiser 只有最后的一些数据，比如累计曝光和累计触达。这篇文章提出了一种方法，能通过最后的数据，估计完整的 reach curve 的曲线。

## Introduction

* $k+$ Reach：$r_k = \frac{\#(Audience_1 \bigcup Audience_2 \bigcup ... \bigcup Audience_k)}{\#population}$
* Rating：$r = \frac{\# Audience}{\#population}$
* GRPs：$GRPs= \sum_i^k r_i = \frac{\#impressions}{\#population} = total~audience \times Averge~Frequency = \frac{\#(Audience_1 \bigcup Audience_2 \bigcup ... \bigcup Audience_k)}{\#population} \times \frac{\#impressions}{\#(Audience_1 \bigcup Audience_2 \bigcup ... \bigcup Audience_k)}$

reach curve 被定义成 GPRs 和 Reach 的函数关系：$r_k(g)$，其中有两个点是已知的：

* $r(0)=0$，对应于没有投广告的时候，自然有 $Reach=0$

* $r_k(G)=R, R \in [0, 100]$，对应于广告投放结束的时候，可以计算出最终的 Reach，$G$ 即为最终的 GRPs。

如果假设 GRPs 与 Reach 存在线性关系，那么就可以计算出 reach curve 是 $r_k(g) = \frac{R}{G}g$，在这种情况下每一次 GRP 的边际 reach 等于 $\frac{R}{\#GRPs}$，只使用线性关系无法很好的估计在点 $g=G$ 的时候边际 reach。

可以在 $g$ 非常小的时候使用线性关系，即 $r_k(g) \approx g \cdot \iota$，其中 $\iota=\iota(k)$ 是第一个看到 $k$ 次广告的人，需要的期望曝光数的倒数。$\iota(k) = \frac{1}{k + log_2k}$ 在实际中常是一个很好的拟合结果。

做一个简单的分析，令 $k = 1$，第一个看到广告的人就只需要 1 次曝光，那么 $\iota(1) = 1$，而当 $k > 1$ 时，第一个看到的 $k$ 次广告的人，需要的曝光次数为 $m$ 次，必然有 $m \geq k$，则有 $\iota(k) = \frac1m \leq \frac1k$。

根据之前的公式 $r_k(g) \approx g \cdot \iota$，可得： $\lim_{G \rightarrow 0}\frac{\partial}{\partial g}r_k(g = G) = \iota$，简单来说，就是在较小的 g 的情况下，可以直接用 $\frac RG = \iota$。


## Reach Curve Models

令 $X \geq 0$ 是曝光数，那么 $k+$ Reach 有：

$
\begin{align*}
& \mathbb{P}(X \geq k, reachable) = \mathbb{P}(X \geq k | reachable) \cdot \mathbb{P}(reachable) \\
& \Leftrightarrow r_k = p_k \cdot \rho
\end{align*}
$

其中 $\rho \in [0, 100]$ 是最大可能触达的概率，$p_k \in [0,1]$ 是在给定实际触达的情况下，触达 $k$ 次的概率。这样分解了之后，两个部分可以用不同的概率模型来表达。

### Gamma-Mixture

2012 年的一篇文章，给出了一个模型：$p_k(g) = 1 - \frac{\beta}{g+\beta}$。文章说，这个模型的 $\beta$ 是 Poisson 分布参数频率 $\lambda$ 上的指数先验，可是翻了原文，也没见到原文中提到了 Poisson 分布了，倒是提出了一个估计 $reach$ 的模型 $reach(g) = a - \frac{b}{g + b / a}$。

Poisson 分布：$P(X=k|\lambda)=\frac{\lambda^k}{k!}e^{- \lambda},\text{ k=0,1,2....}$，其中 $\lambda$ 是单位时间。

这样计算下来，$g$ 的边际 reach 率是 $\frac{\partial}{\partial g}p_k(g)=\frac{\beta}{(g+\beta)^2}$，则有 $\lim_{g \rightarrow 0}\frac{\partial}{\partial g}r_k(g)=\frac{\rho}{\beta}$

### Conditional Logit

本文给出了另一个模型：$logit(p_k(g))=\beta_0 + \beta_1 \cdot log~g$，其中 $logit(p_k(g)) = log~\frac{p_k(g)}{1-p_k(g)} = \beta_0 + \beta_1 \cdot log~g$

这样就可以计算得到 $p_k(g) = 1 - \frac{e^{-\beta_0}}{e^{-\beta_0} + g^{\beta_1}}$，和上面的 Gamma-Mixture 对应下来，$\beta_1=1$，则这个模型变成了 $p_k(g) = 1 - \frac{e^{-\beta_0}}{e^{-\beta_0} + g}$

这里的 Logit 条件模型，以 Gamma-Mixture 的基线，如果 $\beta_1 > 1$，说过每一个 GRPs 更有效率，如果 $\beta_1 = 1$，说明每一个 GRPs 等于基线的效果，如果 $\beta_1 < 1$，说明每一个 GRPs 的花费不达期望。同样的可以计算对应的边际 reach 率：

$\frac{\partial}{\partial g}p_k(g)=e^{\beta_0} \beta_1 \frac{g^{\beta_1 - 1}}{(e^{\beta_0}+ g^{\beta_1})^2}$

$
lim_{g \rightarrow 0} r_k(g) = \left \{
\begin{align*}
& + \infty & \text{if } \beta_1 < 1 \\
& \frac{\rho}{e^{\beta_0}} & \text{if } \beta_1 = 1 \\
& 0 & \text{if } \beta_1 > 1 \\
\end{align*}
\right.
$

## Methodology

上面的两个模型，都只有 1 个参数，因为另一个参数都被设定成了 1($\beta_1 = 1$ 和 $\alpha = 1$)，这样 reach curve 曲线的公式就只有俩参数：

$r(g;\rho, \beta)=\rho(1 - \frac{\beta}{g + \beta})=\rho\frac{g}{g + \beta} \in [0, \rho]$。$\beta$ 参数控制了 GRPs 的效率，较小的 $\beta$ 就使得 reach 增长的快，较大的 $\beta$ 就相应的使得 reach 增长的慢。

$r'(g;\rho, \beta) = \rho \frac{\beta}{(\beta + g)^2}$，并且在 $g = 0$ 的时候，$r'(0) = \frac{\rho}{\beta}$，又由于在较小的 G 的时候，$\frac{\rho}{\beta} = \iota \Rightarrow \rho = \beta \cdot \iota$。同时，由于有最后的 G 和 R 的值，那么 $\rho \frac{G}{\beta + G} = R \Rightarrow \rho = \frac{R(G+ \beta)}{G}$。

联立两式，就可以得到：

$\hat{\rho} = min(\frac{G \cdot R}{G - R / \iota}, 100)$

$\hat{\beta} = \left \{
\begin{align*}
& \frac{\hat{\rho}}{\iota} =  \frac{G \cdot R}{G - R / \iota}, \text{ if } \rho < 100 \\
& G \frac{100 - R}{R}, \text{ if } \rho = 100
\end{align*}
\right.$

### $\hat{\rho} < 100$

代入上面的计算结果：

$r(g) = \frac{G \cdot R \cdot g}{(G - g) \cdot R / \iota + g \cdot G}$，对应的边际 reach：

$r'(g) = \frac{1}{\iota}(\frac{G \cdot R}{(G - g) \cdot R / \iota + g \cdot G})^2$

在最终的点 $g=G$，有：

$r'(G)=\frac{1}{\iota}(\frac{R}{G})^2$

上面说了每一个 GRPs 可以增加的 reach 的值，那么它的倒数，就是每新增一个 reacn，需要的 GPRs 的量。如果广告的总花费是 C，那么增加一个 reach 的花费是：

$\iota\frac{C}{R}\frac{G}{R}$

###  $\hat{\rho} = 100$

类似于之前的做法，把相应的值代入得到：

$r(g) = \frac{g \cdot R}{(G - g) \cdot R / 100 + G}$

$r'(g)=\frac{G \cdot (1 - R / 100) \cdot R}{(G + (g- G) \cdot R / 100)^2}$

$r'(g=G)=\frac{R}{G}(1-\frac{R}{100})$

$c'(r)=\frac{G}{R}\frac{1}{1-R/100}\times \frac{C}{G}$